
# Multi-Agent AI Campaign Planning System on AWS

This notebook provides comprehensive guidance and code to deploy a multi-agent AI campaign planning system using AWS (Bedrock, Lambda, DynamoDB, Step Functions, S3, and Kendra). It includes realistic dummy data and configurations to handle both Gen-Z and enterprise use cases.


## 1. Setup AWS Environment

In [ ]:

import os

os.environ['AWS_ACCESS_KEY_ID'] = 'YOUR_AWS_ACCESS_KEY'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'YOUR_AWS_SECRET_KEY'
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'  # AWS region with Bedrock support


### Install Dependencies

In [ ]:

!pip install boto3


## 2. Provision AWS Resources
### 2.1 Create S3 Buckets

In [ ]:

import boto3, json, random, string

region = os.environ.get('AWS_DEFAULT_REGION', 'us-east-1')
s3 = boto3.client('s3', region_name=region)

suffix = ''.join(random.choice(string.ascii_lowercase + string.digits) for _ in range(6))
briefs_bucket = f"ai-campaign-briefs-{suffix}"
assets_bucket = f"ai-campaign-assets-{suffix}"

for bucket in [briefs_bucket, assets_bucket]:
    s3.create_bucket(Bucket=bucket, CreateBucketConfiguration={'LocationConstraint': region})
    print(f"Created bucket: {bucket}")


### 2.2 Create DynamoDB Table

In [ ]:

dynamodb = boto3.client('dynamodb', region_name=region)

table_name = f"AI_CampaignData_{suffix}"
dynamodb.create_table(
    TableName=table_name,
    AttributeDefinitions=[
        {"AttributeName": "campaign_id", "AttributeType": "S"},
        {"AttributeName": "step", "AttributeType": "S"}
    ],
    KeySchema=[
        {"AttributeName": "campaign_id", "KeyType": "HASH"},
        {"AttributeName": "step", "KeyType": "RANGE"}
    ],
    BillingMode='PAY_PER_REQUEST'
)
dynamodb.get_waiter('table_exists').wait(TableName=table_name)
print(f"DynamoDB Table created: {table_name}")


### 2.3 Upload Dummy Brief to S3

In [ ]:

campaign_brief = {
    "campaign_id": "CAMPAIGN123",
    "campaign_name": "GenZ Social Hype 2025",
    "target_audience": "Gen-Z",
    "product": "Tech Gadget X",
    "objectives": "Drive brand engagement among Gen-Z.",
    "key_messages": "Innovative, trendy, community-driven.",
    "channels": "social_media",
    "timeline": "Q3 2025",
    "budget": "50k USD"
}

brief_key = "sample_brief_genz.json"
s3.put_object(Bucket=briefs_bucket, Key=brief_key, Body=json.dumps(campaign_brief))
print(f"Uploaded brief to s3://{briefs_bucket}/{brief_key}")


## 3. Validate Setup

In [ ]:

print("Buckets:", [bucket['Name'] for bucket in s3.list_buckets()['Buckets']])
print("DynamoDB Tables:", dynamodb.list_tables()['TableNames'])



## Next Steps

After completing the above setup, continue to implement:

- Lambda functions for each AI agent (Input Extractor, Business Interpreter, Planner, Content Generator, Channel Strategist, Evaluator).
- AWS Step Functions workflow orchestration.
- Amazon Kendra for a knowledge base.
- Amazon SNS for notifications.
- IAM roles and permissions.

Ensure you handle AWS permissions correctly. Replace placeholders with secure methods like IAM roles.



## Next Steps

With your basic AWS setup validated, proceed by implementing the following steps to fully deploy your multi-agent AI Campaign Planner:

### 4. Create and Deploy Lambda Functions
Implement the following AWS Lambda functions using `boto3` and Amazon Bedrock for generative tasks:

- **Agent 0: Input Extractor** - Extracts campaign details from briefs.
- **Agent 1: Business Interpreter** - Analyzes context using Kendra knowledge base.
- **Agent 2: Campaign Planner** - Plans campaign strategy.
- **Agent 3: Content Generator** - Creates targeted campaign content.
- **Agent 4: Channel Strategist** - Recommends distribution channels.
- **Agent 5: Evaluator** - Reviews and assesses campaign effectiveness.

### 5. Set Up AWS Step Functions Workflow
Configure AWS Step Functions to orchestrate Lambda agents:

- Create a state machine that sequences Lambda executions.
- Include error handling and retry logic.
- Add SNS notifications for campaign success or failure alerts.

### 6. Set Up Amazon Kendra Knowledge Base
- Create a Kendra index and ingest dummy data relevant to Gen-Z and Enterprise marketing strategies.
- Configure Lambda agents to query Kendra.

### 7. Configure Amazon SNS Notifications
- Set up SNS topics for notifications on campaign status.
- (Optional) Subscribe endpoints (e.g., email, SMS).

### 8. Configure IAM Roles and Permissions
- Ensure least privilege access for Lambda, Step Functions, S3, DynamoDB, Kendra, and SNS.
- Validate IAM policies and roles carefully.

### 9. Test and Monitor the Entire Workflow
- Execute the workflow with sample campaign briefs.
- Monitor via CloudWatch Logs and Step Functions console.

### 10. Cleanup (optional)
- After tests, consider cleaning up AWS resources to prevent unnecessary charges.

By following these steps, you'll achieve a fully automated, scalable, and production-ready multi-agent AI campaign planning system.
